In [ ]:

def richStat(data, catalog, r_low, r_high, bins = 1, save_plot = False):
    ra = catalog[1].data['RADeg']
    names = catalog[1].data['name']
    dec = catalog[1].data['decDeg']
    ra, dec = np.array(ra), np.array(dec)
    rich = catalog[1].data['Rich']
    names = catalog[1].data['name']
    path = '/scratch/r/rbond/jorlo/S18d_202006/filteredMaps/'

    cut = np.where((r_low<np.array(rich)) & (np.array(rich)<=r_high))[0]
    cut_ra, cut_dec, cut_names, cut_rich = ra[cut], dec[cut], names[cut], rich[cut]
    
    if bins == 1:
        stack, num = s18dStack(cut_ra, cut_dec, data)

        plot = plt.imshow(stack)
        plt.colorbar(plot, format='%.0e')
        plt.title("ACT S18 dStack on MaDCoWS w/ {}=>Richness>{}\n Total in stack: {}".format(r_high, r_low, num))

        plt.show()
        mean = np.amax(stack[18:22, 18:22])

        brights = []
        for i in range(len(cut_ra)):
            stamp = s18dStamp(cut_ra[i], cut_dec[i], s18d, 'fakename', width = 1/3, write = False)
            if stamp is None: continue
            brights.append(np.amax(stamp[0][18:22, 18:22]))

            if save_plot:
                plot = plt.imshow(stamp[0], extent = [-20,20,-20,20])
                plt.scatter(0,0, marker = '+', color = 'r')
                plt.colorbar(plot, format='%.0e')
                plt.title('Cluster {}, {}th Percentile'.format(cut_names[i], round(100/(bin))))

                plt.savefig('plots/indv_clusters/' + str(cut_names[i]) + '.png')
                plt.close()

        var = np.var(brights)
        #print(brights)
    
        return mean, var
    else:
        percs = [r_low]
        means = []
        varies =[]
        #NOTE: Percentiles are in terms of the full MaDCoWS catalog: these will not neccesarily be exactly the
        #percentiles for the subset of the clusters that are in the s18d footprint
        for i in range(bins):
            perc = np.percentile(cut_rich, (i+1)*100/(bins))
            percs.append(perc)
        for i in range(bins):
            flags2 = np.where((percs[i] < cut_rich) & (cut_rich <= percs[i+1]))[0]
            cut_ra2, cut_dec2, cut_names2, cut_rich2 = cut_ra[flags2], cut_dec[flags2], cut_names[flags2], cut_rich[flags2]
        
            stack, num = s18dStack(cut_ra2, cut_dec2, data)

            plot = plt.imshow(stack, extent = [-20,20,-20,20])
            plt.colorbar(plot, format='%.0e')
            plt.scatter(0,0, marker = '+', color = 'r')
            
            plt.title("ACT S18d Stack on MaDCoWS w/ {}=>Richness>{}\n {} to {}th percentile, total in stack: {}".format(r_high, r_low, int((i)*100/(bins)), int((i+1)*100/(bins)), num))
            plt.savefig('plots/rich_bins/rich_bin_{}_{}_{}_rich_perc.png'.format(r_low, r_high, int((i+1)*100/(bins))))
            plt.show()
            mean = np.amax(stack[18:22, 18:22])

            brights = []
            for i in range(len(cut_ra2)):
                stamp = s18dStamp(cut_ra2[i], cut_dec2[i], s18d, 'fakename', width = 1/3, write = False)
                if stamp is None: continue
                brights.append(np.amax(stamp[0][18:22, 18:22]))

                if save_plot:
                    plot = plt.imshow(stamp[0], extent = [-20,20,-20,20])
                    plt.scatter(0,0, marker = '+', color = 'r')
                    plt.colorbar(plot, format='%.0e')
                    plt.title('Cluster {}, {}th Richness Percentile'.format(cut_names2[i], int((i+1)*100/(bins))))
                    
                    plt.savefig('plots/indv_clusters/' + str(cut_names2[i]) + '_richness_' + str(cut_rich2[i]) + '_perc_' + str(int((i+1)*100/(bins))) + '.png')
                    plt.close()

            var = np.var(brights)
            varies.append(np.sqrt(var))
            means.append(mean)
            
        return means, varies
    

In [ ]:
mdcw_catalog = fits.open('/home/s/sievers/sdicker/ACTnCOWs/MADCOWSUnion.fits')

means = []
for i in range(0, 70, 10):
    mean, ignore = richStat(s18d, mdcw_catalog, i, i + 10)
    means.append(mean)
    print(np.sqrt(ignore))

In [ ]:
long_means =[]
long_vars = []
for i in range(0, 60, 10):
    mean, var = richStat(s18d, mdcw_catalog, i, i+10, bins = 4, save_plot = True)
    long_means.append(mean)
    long_vars.append(var)

In [ ]:
temp_var = np.transpose(long_vars)
temp = np.transpose(long_means)
for i in range(len(temp)):
    plt.errorbar( range(3+i,63+i, 10),temp[i], yerr= temp_var[i], label = str(int((i+1)*100/(len(temp))))+'th Percentile', marker='o',linestyle="dashed")
plt.legend()
plt.xlabel('Richness')
plt.xticks(range(5, 65, 10))
plt.ylabel('Brightest Pixel')
plt.title('Richness v. Brightest Pixel, \n Bins of 10 Richnees, Percentiles of Richness')
plt.savefig('richness-brightness.png')
plt.show()

In [ ]:
#Now still binning in Richness but in percentiles of brightest pixel (center 4x4 pixels)
def richStatBright(data, catalog, r_low, r_high, bins = 1, save_plot = False):
    ra = catalog[1].data['RADeg']
    names = catalog[1].data['name']
    dec = catalog[1].data['decDeg']
    ra, dec = np.array(ra), np.array(dec)
    rich = catalog[1].data['Rich']
    names = catalog[1].data['name']
    path = '/scratch/r/rbond/jorlo/S18d_202006/filteredMaps/'

    cut = np.where((r_low<np.array(rich)) & (np.array(rich)<=r_high))[0]
    cut_ra, cut_dec, cut_names, cut_rich = ra[cut], dec[cut], names[cut], rich[cut]
    
    stack, num = s18dStack(cut_ra, cut_dec, data)

    #plot = plt.imshow(stack)
    #plt.colorbar(plot, format='%.0e')
    #plt.title("ACT S18 dStack on MaDCoWS w/ {}=>Richness>{}\n Total in stack: {}".format(r_high, r_low, num))

    #plt.show()
    #mean = np.amax(stack[18:22, 18:22])

    brights = []
    for i in range(len(cut_ra)):
        stamp = s18dStamp(cut_ra[i], cut_dec[i], s18d, 'fakename', width = 1/3, write = False)
        if stamp is None: continue
        brights.append(np.amax(stamp[0][18:22, 18:22]))



    
    percs = [0]
    means = []
    varies = []
    #NOTE: Percentiles are in terms of the full MaDCoWS catalog: these will not neccesarily be exactly the
    #percentiles for the subset of the clusters that are in the s18d footprint
    for i in range(bins):
        perc = np.percentile(brights, (i+1)*100/(bins))
        percs.append(perc)
        
    for i in range(bins):
        flag = np.where((percs[i] < np.array(brights)) & (np.array(brights) <= percs[i+1]))[0]
        
        cut_ra2, cut_dec2, cut_names2, brights2 = cut_ra[flag], cut_dec[flag], cut_names[flag], np.array(brights)[flag]
        
        stack, num = s18dStack(cut_ra2, cut_dec2, data)
        
        plot = plt.imshow(stack, extent = [-20,20,-20,20])
        plt.scatter(0,0, marker = '+', color = 'r')
        plt.colorbar(plot, format='%.0e')
        plt.title("ACT S18d Stack on MaDCoWS w/ {}=>Richness>{}\n {} to {}th brightness percentile, total in stack: {}".format(r_high, r_low, int((i)*100/(bins)), int((i+1)*100/(bins)), num))
        plt.savefig('plots/rich_bins/rich_bin_{}_{}_{}_bright_perc.png'.format(r_low, r_high, int((i+1)*100/(bins))))
        plt.show()
        plt.close()

        means.append(np.amax(stack[18:22, 18:22]))
        var = np.var(brights2)
        varies.append(np.sqrt(var))
        
    return means, varies

In [ ]:
long_means2 = []
long_vars2 = []
for i in range(0, 60, 10):
    mean, var = richStatBright(s18d, mdcw_catalog, i, i+10, bins = 4)
    long_means2.append(mean)
    long_vars2.append(var)
    

In [ ]:
temp2 = np.transpose(long_means2)
temp_var2 = np.transpose(long_vars2)
for i in range(len(temp)):
    plt.errorbar( range(3+i,63+i, 10),temp2[i], yerr = temp_var2[i], label = str(int((i+1)*100/(len(temp))))+'th Percentile', marker='o',linestyle="dashed")
plt.legend()
plt.xlabel('Richness')
plt.xticks(range(5, 65, 10))
plt.ylabel('Brightest Pixel')
plt.title('Richness v. Brightest Pixel, \n Bins of 10 Richnees, Percentiles of Brightness')
plt.savefig('richness-brightness2.png')
plt.show()